In [1]:
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix

In [204]:
def prepare_data(n, model=None, start=False, n_bids=100):

    ############ FIRST ROUND

    # valuations and actions of the player
    r1_remain_player   = np.random.beta(1,1,n)
    r1_action_player  = np.random.beta(1,1,n)
    r1_remain_opponent = np.random.beta(1,1,n)

    # if first iteration - opponent plays randomly
    if start==True:
        r1_action_opponent = np.random.beta(1,1,n)
    else:
        # opponent chooses optimal bid given his valuation
        X_r1_opponent = np.concatenate((np.repeat(0, n).reshape(n,1),
                                        np.repeat(0, n).reshape(n,1),
                                        (r1_remain_opponent + np.random.beta(1,1,n)/1000).reshape(n,1),
                                        np.repeat(0, n).reshape(n,1),
                                        np.repeat(0, n).reshape(n,1)), axis=1)
        
        X_r1_opponent = np.repeat(X_r1_opponent, n_bids, axis=0)
        action_opponent = np.tile(np.linspace(start=0, stop=1, num=n_bids), n)
        
        # gather private values and bids together
        X_r1_opponent = np.concatenate((X_r1_opponent, action_opponent.reshape(n*n_bids,1)), axis=1)
        
        # determine optimal bid
        X_r1_opponent = opt_bid(model, X_r1_opponent)
        r1_remain_opponent = X_r1_opponent[:,0]
        r1_action_opponent = X_r1_opponent[:,1]
        print("# Determined opt bids R1")
    
    # determine bids and winner of the first round
    r1_bid_player   = r1_remain_player*r1_action_player
    r1_bid_opponent = r1_remain_opponent*r1_action_opponent
    r1_winner = (r1_bid_player>r1_bid_opponent)
    print("# End of R1")

    ############ SECOND ROUND
    r2_remain_player = r1_remain_player - r1_bid_player
    r2_remain_opponent = r1_remain_opponent - r1_bid_opponent

    #### CASE 1. Player lost R1 and Opponents r1_bid_opponent > r1_remain_player


    #### CASE 2.Player lost R1 and Opponents r1_bid_opponent < r1_remain_player

    # bid if lost first round
    r2_act_lost_player = np.random.beta(1,1,n)
    if start==True:
        r2_act_lost_opponent = np.random.beta(1,1,n)
    else:
        # remeber order of r2_remain_opponent to sort all other arrays
        r2_remain_opponent = r1_remain_opponent-r1_bid_opponent
        r2_sort = np.argsort(r2_remain_opponent)
        
        # opponent chooses optimal bid given his valuation and bid of the player
        X_r2_opponent = np.concatenate((np.repeat(1, n).reshape(n,1),
                                        r1_bid_opponent.reshape(n,1),
                                        (r2_remain_opponent + np.random.beta(1,1,n)/1000).reshape(n,1),
                                        np.repeat(0, n).reshape(n,1),
                                        r1_bid_player.reshape(n,1)), axis=1)
        
        X_r2_opponent = np.repeat(X_r2_opponent, n_bids, axis=0)
        action_opponent = np.tile(np.linspace(start=0, stop=1, num=n_bids), n)
        
        # gather private values and bids together
        X_r2_opponent = np.concatenate((X_r2_opponent, action_opponent.reshape(n*n_bids,1)), axis=1)
        
        # determine optimal bid
        X_r2_opponent = opt_bid(model, X_r2_opponent)
        r2_remain_opponent = X_r2_opponent[:,0]
        r2_act_lost_opponent = X_r2_opponent[:,1]
        
        # sort other arrays
        r1_remain_player = r1_remain_player[r2_sort]
        r1_remain_opponent = r1_remain_opponent[r2_sort]
        r1_bid_opponent = r1_bid_opponent[r2_sort]
        r1_bid_player = r1_bid_player[r2_sort]
        r1_action_player = r1_action_player[r2_sort]
        r1_action_opponent = r1_action_opponent[r2_sort]
        r1_winner = r1_winner[r2_sort]
        r2_act_lost_player = r2_act_lost_player[r2_sort]
        r2_remain_player = r2_remain_player[r2_sort]
        print("# Determined opt bids R2")
        
    
    # condition for case 2
    r2_case2_cond_player = (r1_remain_player>r1_bid_opponent)*(1-r1_winner)
    
    # absolute bid
    r2_bid_lost_player = (r1_bid_opponent-r1_bid_player) + (r1_remain_player-r1_bid_opponent)*r2_act_lost_player
    r2_bid_lost_opponent = (r1_bid_player-r1_bid_opponent) + (r1_remain_opponent-r1_bid_player)*r2_act_lost_opponent

    # player wins after losing 1st round if sum of his bids>private value of opponent
    r2_cond_win_lostr1 = (r1_bid_player+r2_bid_lost_player)>r1_remain_opponent
    profit_lost_r1 = (r2_remain_player - r2_bid_lost_player)*(r2_cond_win_lostr1*r2_case2_cond_player)


    ### CASE 3. Won the first round (r1_bid_opponent+r2_bid_lost_opponent) < r1_bid_player
    r2_case3_cond_player = r1_remain_opponent<r1_bid_player
    profit_win_r1_case3 = r2_remain_player*r2_case3_cond_player

    ### Case 4. Won the first round (r1_bid_opponent+r2_bid_lost_opponent) > r1_bid_player

    # condition for case 3
    r2_case4_cond1_player = ((r1_bid_opponent+r2_bid_lost_opponent)>r1_bid_player)
    r2_case4_cond2_player = ((r1_bid_opponent+r2_bid_lost_opponent)<r1_remain_player)

    # player sees both bids of the opponent and only adds marginal value to win the auction
    profit_win_r1_case4 = (r1_remain_player - r1_bid_opponent - r2_bid_lost_opponent - 0.0000001)*r2_case4_cond1_player*r2_case4_cond2_player*r1_winner

    ### Final value of profit for player
    profit = profit_lost_r1 + profit_win_r1_case3 + profit_win_r1_case4
    
    res = [r1_remain_player, r1_action_player, r1_bid_player, r1_bid_opponent, r1_winner, 
    r2_remain_player, r2_act_lost_player, r2_case2_cond_player, profit]
    
    return res

In [165]:
def prepare_data_fit(r1_remain_player, r1_action_player, r1_bid_player, r1_bid_opponent, r1_winner,
                     r2_remain_player, r2_act_lost_player, r2_case2_cond_player, profit):


    X_r1 = np.concatenate((np.repeat(0, n).reshape(n,1),
                           np.repeat(0, n).reshape(n,1),
                           r1_remain_player.reshape(n,1),
                           np.repeat(0, n).reshape(n,1),
                           np.repeat(0, n).reshape(n,1),
                           r1_action_player.reshape(n,1),
                           profit.reshape(n,1)), axis=1)
                       
    X_r2 = np.concatenate((np.repeat(1, n).reshape(n,1),
                           r1_bid_player.reshape(n,1),
                           r2_remain_player.reshape(n,1),
                           r1_winner.reshape(n,1),
                           r1_bid_opponent.reshape(n,1),
                           r2_act_lost_player.reshape(n,1),
                           profit.reshape(n,1),
                           r2_case2_cond_player.reshape(n,1)), axis=1)

    # keep only those instances from R2 where r2_case2_cond_player=True
    X_r2 = X_r2[X_r2[:,7]==1,0:7]

    # concatenate results of both rounds
    X = np.concatenate((X_r1, X_r2), axis=0)

    # separate profit
    y = X[:,-1]
    X = X[:,0:6]
    
    return X, y

In [166]:
def fit_model(X, y):
    
    # create a neural network
    mlp = MLPRegressor(alpha=0.001, hidden_layer_sizes = (10,5,3), max_iter = 1000, 
                   activation = 'logistic', verbose = True, learning_rate = 'adaptive')
    # fit model
    mlp.fit(X, y)
    
    return mlp

In [167]:
def track_progress(model):
    n_val = 5
    n_bids = 100
    n_total = n_val*n_bids
    
    ### First round 
    X_r1 = np.concatenate((np.repeat(0, n_val).reshape(n_val,1),
                        np.repeat(0, n_val).reshape(n_val,1),
                        np.array([0.1, 0.3, 0.5, 0.7, 0.9]).reshape(n_val,1),
                        np.repeat(0, n_val).reshape(n_val,1),
                        np.repeat(0, n_val).reshape(n_val,1)), axis=1)

    # create private values and bids
    X_r1 = np.repeat(X_r1, n_bids, axis=0)
    action_player = np.tile(np.linspace(start=0, stop=1, num=n_bids), n_val)

    # gather private values and bids together
    X_r1 = np.concatenate((X_r1, action_player.reshape(n_total,1)), axis=1)
    
    X_r1 = opt_bid(model, X_r1)
    
    ### Second round
    X_r2 = np.concatenate((np.repeat(1, n_val).reshape(n_val,1),
                          (X_r1[:,0]*X_r1[:,1]).reshape(n_val,1),
                          (X_r1[:,0] - X_r1[:,0]*X_r1[:,1]).reshape(n_val,1),
                           np.repeat(0, n_val).reshape(n_val,1),
                          (X_r1[:,0]*X_r1[:,1] + (X_r1[:,0] - X_r1[:,0]*X_r1[:,1])*0.5).reshape(n_val,1)), axis=1)
                          
    X_r2 = np.repeat(X_r2, n_bids, axis=0)
    X_r2 = np.concatenate((X_r2, action_player.reshape(n_total,1)), axis=1)
    
    X_r2 = opt_bid(model, X_r2)
    
    return X_r1, X_r2

def opt_bid(model, X):
    '''Find bid with the highest expected profit'''
    
    # predict expected profit for each pair
    cand_predict = model.predict(X)
    cand_predict[cand_predict <= 0] = 0
    cand_predict[cand_predict >= 1] = 1

    # add expected profit to cand
    cand_res = np.concatenate((X, cand_predict.reshape(X.shape[0],1)), axis=1)

    # sort cand_res in descending order by the expected profit
    cand_res = cand_res[cand_res[:,-1].argsort()[::-1],:]

    # keep only best bids
    unique_keys, indices = np.unique(cand_res[:,2], return_index=True)
    cand_res = cand_res[indices,:]
    
    
    return cand_res[:,[2,5,6]]

In [205]:
#### Run algorithm

np.random.seed(5)
n = 1000000

vector_opt_bids_r1 = np.array([0.1, 0.3, 0.5, 0.7, 0.9]).reshape(5,1)
vector_opt_bids_r2 = np.array([0.1, 0.3, 0.5, 0.7, 0.9]).reshape(5,1)

for iter in range(20):
    
    print("########## Iteration: ", iter, "\n")
    
    # prepare data
    if iter==0:
        inputs = prepare_data(n, model=None, start=True)
    else:
        inputs = prepare_data(n, model=mlp, start=False)
        # save this model
        mlp_prev = mlp

    X, y = prepare_data_fit(inputs[0],inputs[1],inputs[2],inputs[3],inputs[4],inputs[5],
                            inputs[6],inputs[7],inputs[8])

    mlp = fit_model(X, y)

    # print optimal bids
    check_R1, check_R2 = track_progress(mlp)
    vector_opt_bids_r1 = np.concatenate((vector_opt_bids_r1, check_R1[:,1].reshape(5,1)), axis=1)
    vector_opt_bids_r2 = np.concatenate((vector_opt_bids_r2, check_R2[:,1].reshape(5,1)), axis=1)

    print("First round:")
    print(vector_opt_bids_r1[:,[0,-1]])

    print("Second round:")
    print(vector_opt_bids_r2[:,[0,-1]])
    
    # if there are extreme values in vector_opt_bids - repeat the iteration
    if (np.sum(vector_opt_bids_r1[:,-1]==0) + np.sum(vector_opt_bids_r1[:,-1]==1))!=0:
        print("Repeating iteration. Extreme values in 1st round")
        mlp = mlp_prev
    elif (np.sum(vector_opt_bids_r2[:,-1]==0) + np.sum(vector_opt_bids_r2[:,-1]==1))!=0:
        print("Repeating iteration. Extreme values in 2nd round")
        mlp = mlp_prev

    del X,y


########## Iteration:  0 

# End of R1
Iteration 1, loss = 0.00926542
Iteration 2, loss = 0.00828816
Iteration 3, loss = 0.00806896
Iteration 4, loss = 0.00772588
Iteration 5, loss = 0.00766246
Iteration 6, loss = 0.00764256
Iteration 7, loss = 0.00762859
Iteration 8, loss = 0.00761596
Iteration 9, loss = 0.00760509
Iteration 10, loss = 0.00759514
Iteration 11, loss = 0.00758554
Iteration 12, loss = 0.00757821
Iteration 13, loss = 0.00757001
Iteration 14, loss = 0.00756409
Iteration 15, loss = 0.00755772
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
First round:
[[0.1        0.45454545]
 [0.3        0.4040404 ]
 [0.5        0.37373737]
 [0.7        0.34343434]
 [0.9        0.32323232]]
Second round:
[[0.1        0.8989899 ]
 [0.3        0.52525253]
 [0.5        0.51515152]
 [0.7        0.51515152]
 [0.9        0.51515152]]
########## Iteration:  1 

# Determined opt bids R1
# End of R1
# Determined opt bids R2
Iteration 1, loss = 0.01640925
I

Iteration 1, loss = 0.00813282
Iteration 2, loss = 0.00760681
Iteration 3, loss = 0.00705968
Iteration 4, loss = 0.00664201
Iteration 5, loss = 0.00655332
Iteration 6, loss = 0.00651551
Iteration 7, loss = 0.00649188
Iteration 8, loss = 0.00647305
Iteration 9, loss = 0.00645759
Iteration 10, loss = 0.00644532
Iteration 11, loss = 0.00643279
Iteration 12, loss = 0.00642021
Iteration 13, loss = 0.00640981
Iteration 14, loss = 0.00640091
Iteration 15, loss = 0.00639243
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
First round:
[[0.1        0.51515152]
 [0.3        0.46464646]
 [0.5        0.42424242]
 [0.7        0.4040404 ]
 [0.9        0.39393939]]
Second round:
[[0.1        0.54545455]
 [0.3        0.66666667]
 [0.5        0.        ]
 [0.7        0.75757576]
 [0.9        1.        ]]
Repeating iteration. Extreme values in 2nd round
########## Iteration:  9 

# Determined opt bids R1
# End of R1
# Determined opt bids R2
Iteration 1, loss = 0.

# Determined opt bids R1
# End of R1
# Determined opt bids R2
Iteration 1, loss = 0.00954262
Iteration 2, loss = 0.00741070
Iteration 3, loss = 0.00678779
Iteration 4, loss = 0.00668851
Iteration 5, loss = 0.00665341
Iteration 6, loss = 0.00662724
Iteration 7, loss = 0.00660771
Iteration 8, loss = 0.00658973
Iteration 9, loss = 0.00657460
Iteration 10, loss = 0.00656047
Iteration 11, loss = 0.00654811
Iteration 12, loss = 0.00653851
Iteration 13, loss = 0.00652958
Iteration 14, loss = 0.00652164
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
First round:
[[0.1        0.51515152]
 [0.3        0.45454545]
 [0.5        0.42424242]
 [0.7        0.4040404 ]
 [0.9        0.39393939]]
Second round:
[[0.1        0.52525253]
 [0.3        0.75757576]
 [0.5        0.        ]
 [0.7        0.75757576]
 [0.9        1.        ]]
Repeating iteration. Extreme values in 2nd round
########## Iteration:  18 

# Determined opt bids R1
# End of R1
# Determined opt

KeyboardInterrupt: 

In [206]:
np.round(vector_opt_bids_r1,3)

array([[0.1  , 0.455, 0.495, 0.606, 0.545, 0.556, 0.485, 0.556, 0.475,
        0.515, 0.556, 0.505, 0.535, 0.535, 0.515, 0.515, 0.545, 0.525,
        0.515, 0.545],
       [0.3  , 0.404, 0.434, 0.455, 0.434, 0.444, 0.444, 0.475, 0.404,
        0.465, 0.485, 0.465, 0.485, 0.465, 0.455, 0.465, 0.465, 0.475,
        0.455, 0.465],
       [0.5  , 0.374, 0.404, 0.404, 0.414, 0.414, 0.414, 0.424, 0.364,
        0.424, 0.434, 0.434, 0.434, 0.424, 0.424, 0.434, 0.424, 0.434,
        0.424, 0.434],
       [0.7  , 0.343, 0.394, 0.384, 0.394, 0.394, 0.404, 0.404, 0.364,
        0.404, 0.414, 0.414, 0.404, 0.404, 0.404, 0.414, 0.404, 0.414,
        0.404, 0.414],
       [0.9  , 0.323, 0.384, 0.374, 0.384, 0.384, 0.394, 0.394, 0.384,
        0.394, 0.394, 0.394, 0.384, 0.404, 0.394, 0.394, 0.394, 0.394,
        0.394, 0.404]])

In [208]:
np.savetxt("vector_opt_bids_r1.csv", vector_opt_bids_r1, delimiter=",")

In [209]:
np.savetxt("vector_opt_bids_r2.csv", vector_opt_bids_r2, delimiter=",")